In [1]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import makeNaNImputationTest

inputType ='influx' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
"""
imputation_param ={
"imputation_method":[
    {"min":0,"max":1,"method":"mean"},
    {"min":2,"max":4,"method":"linear"},
    {"min":5,"max":10,"method":"brits"}],
"totalNanLimit":0.3}
"""
imputation_param ={
"imputation_method":[
    {"min":0,"max":5,"method":"mean"}],
"totalNanLimit":30}
# mean, median X

# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = makeNaNImputationTest.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
input_test_data = input_data.replace(4.0, np.nan)
output = data_preprocessing.get_preprocessed_data(input_test_data[:25], refine_param, outlier_param, imputation_param)

Preprocessing:Refining
Preprocessing:DataWithMoreNaN
Preprocessing:Imputation
Ratio in_ciai 16.0
Ratio in_cici 16.0
Ratio in_cici_co2 28.000000000000004
Ratio in_cici_humi 16.0
Ratio in_cici_noise 12.0
Ratio in_cici_pm10 28.000000000000004
Ratio in_cici_pm25 0.0
Ratio in_cici_temp 12.0
Ratio in_cici_voc 12.0
Ratio in_co2 4.0
Ratio in_humi 4.0
Ratio in_noise 4.0
Ratio in_pm01 8.0
Ratio in_pm01_raw 8.0
Ratio in_pm10 8.0
Ratio in_pm10_raw 8.0
Ratio in_pm25 0.0
Ratio in_pm25_raw 0.0
Ratio in_temp 4.0
Ratio in_voc 4.0


In [3]:
for key in output:
    print(key, output[key][:20])

original                            in_ciai  in_cici  in_cici_co2  in_cici_humi  \
time                                                                     
2021-02-04 10:33:00+00:00     53.0     57.0          0.0          80.0   
2021-02-04 10:34:00+00:00     67.0     60.0         88.0          54.0   
2021-02-04 10:35:00+00:00     69.0     65.0         87.0          50.0   
2021-02-04 10:36:00+00:00     73.0     66.0         88.0          47.0   
2021-02-04 10:37:00+00:00     76.0     69.0         89.0          47.0   
2021-02-04 10:38:00+00:00     82.0     78.0         89.0          50.0   
2021-02-04 10:39:00+00:00     86.0     83.0         89.0          47.0   
2021-02-04 10:40:00+00:00     90.0     87.0         89.0          47.0   
2021-02-04 10:41:00+00:00     91.0     88.0         89.0          47.0   
2021-02-04 10:42:00+00:00     92.0     89.0         89.0          47.0   
2021-02-04 10:43:00+00:00     92.0     89.0         89.0          47.0   
2021-02-04 10:44:00+00:00    

In [ ]:
from KETIPrePartialDataPreprocessing import data_preprocessing

MDP = data_preprocessing.DataPreprocessing()
###########
print("=============")
print('original', input_test_data.isna().sum())
print(input_data[:20])
refined_data = MDP.get_refinedData(input_data, refine_param)
print("=============")
print('after refine', refined_data.isna().sum())
print(refined_data[:20])
###########
datawithMoreCertainNaN, datawithMoreUnCertainNaN = MDP.get_outlierToNaNData(refined_data, outlier_param)
print("=============")
print('after outlierDetection', datawithMoreUnCertainNaN.isna().sum())
print(datawithMoreUnCertainNaN[:20])
###########

### TODO ST Oh
imputed_data = MDP.get_imputedData(datawithMoreUnCertainNaN, imputation_param)
print("=============")
print('after imputation', imputed_data.isna().sum())
print(imputed_data[:20])
    
from KETIPrePartialDataPreprocessing.dataRemovebyNaN import data_remove_byNaN
nanM = data_remove_byNaN.DataRemoveByNaNStatus()
nanMap = nanM.consecutiveNaNCountMap(datawithMoreUnCertainNaN)

In [ ]:
nanMap.isna().sum()

In [ ]:
import pandas as pd 
data = datawithMoreUnCertainNaN
consecutiveNanCountMap = pd.DataFrame() 
column_list = data.columns
for column_name in column_list:
    consecutiveNanCountMap[column_name] = data[column_name].isnull().astype(int).groupby(
        data[column_name].notnull().astype(int).cumsum()).cumsum()

consecutiveNanCountMap.plot()

In [ ]:
consecutiveNanCountMap